# Python HANA ML APL

<div class="alert alert-block alert-info">
<b>Forecasting daily cashflow.</b> <br>
</div>

In [ ]:
%run ./02-setup.ipynb

### Create an HANA Dataframe for the actual series

In [ ]:
sql_cmd = 'SELECT "Date", "Cash", "MondayMonthInd", "FridayMonthInd" FROM "CASHFLOW"'
series_in = hdf.DataFrame(conn, sql_cmd)

series_in.head(5).collect()

### Fit & Predict with APL

In [ ]:
from hana_ml.algorithms.apl.time_series import AutoTimeSeries
apl_model = AutoTimeSeries(time_column_name= 'Date', target= 'Cash', horizon= 21)
series_out = apl_model.fit_predict(data = series_in, build_report=True)

In [ ]:
df = series_out.select(series_out.columns[0:5]).collect()
dict = {'ACTUAL': 'Actual', 
        'PREDICTED_1': 'Forecast', 
        'LOWER_INT_95PCT': 'Lower Limit', 
        'UPPER_INT_95PCT': 'Upper Limit' }
df.rename(columns=dict, inplace=True)
df

##### Reports

In [ ]:
apl_model.generate_html_report('Cash_Report')
apl_model.generate_notebook_iframe_report()

##### Target Statistics

In [ ]:
df = apl_model.get_debrief_report('ContinuousTarget_Statistics').collect()
df.drop('Oid', axis=1, inplace=True)
df.style.hide(axis='index')